In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    os.chdir(project_root)

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

print(f"Using device: {device}")

GPU available: NVIDIA GeForce RTX 3060 Laptop GPU
Number of GPUs: 1
Using device: cuda


In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import os

from langsmith import Client

client = Client()

#### Load document

In [6]:
from langchain_community.document_loaders import PDFPlumberLoader

from paths import DATA_DIR

pdf_loader = PDFPlumberLoader(DATA_DIR / "docs" / "company_overview.pdf")
docs = pdf_loader.load()

docs

[Document(metadata={'source': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'file_path': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'page': 0, 'total_pages': 2, 'Title': 'Untitled document', 'Producer': 'Skia/PDF m139 Google Docs Renderer'}, page_content='Company Name: EchoNova Audio\nDocument Title: Company Overview & Core Values\nDate: June 29, 2025\nCompany Overview\nEchoNova Audio is a multinational retailer and support provider specializing in premium\naudio electronics and related services. We distribute world-class products such as wireless\nheadphones, portable speakers, earbuds, and smart audio gear from top manufacturers\nincluding Sony, Bose, JBL, Apple, and Sennheiser.\nOperating across the United States and key European markets—such as Germany, France,\nthe UK, Netherlands, and Sweden—we serve both individual consumers and corporate\nclients with a seamless customer experience, fast logistics, and industry-recognized

#### Split docs 

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

splits

[Document(metadata={'source': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'file_path': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'page': 0, 'total_pages': 2, 'Title': 'Untitled document', 'Producer': 'Skia/PDF m139 Google Docs Renderer'}, page_content='Company Name: EchoNova Audio\nDocument Title: Company Overview & Core Values\nDate: June 29, 2025\nCompany Overview\nEchoNova Audio is a multinational retailer and support provider specializing in premium\naudio electronics and related services. We distribute world-class products such as wireless'),
 Document(metadata={'source': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'file_path': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'page': 0, 'total_pages': 2, 'Title': 'Untitled document', 'Producer': 'Skia/PDF m139 Google Docs Renderer'}, page_content='headphones, portable speakers, earbuds, and smart audio gear from top man

#### Vector Store

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langsmith import Client

client = Client()

vectorstore = Chroma.from_documents(
    documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
)

/tmp/ipykernel_36157/2516283342.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4}, name="Ticket Agent")

results = retriever.invoke("What is the company name?")

results

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(id='cedf929a-9490-4614-a62f-d3bfeb709b94', metadata={'source': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'total_pages': 2, 'Title': 'Untitled document', 'Producer': 'Skia/PDF m139 Google Docs Renderer', 'file_path': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'page': 0}, page_content='global brands.\n● Paid Services: Extended warranties, device protection plans, music service bundles,\nand setup assistance.\n● Customer Support: Specialized departments for Tech Support, Billing, Shipping,\nLegal, Sales, and Customer Care.'),
 Document(id='c2e9c302-8cf2-4527-bd37-f68f9af4903e', metadata={'source': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'total_pages': 2, 'page': 0, 'file_path': '/home/tufman/src/personal/ticket-agent/data/docs/company_overview.pdf', 'Producer': 'Skia/PDF m139 Google Docs Renderer', 'Title': 'Untitled document'}, page_content='Company Name: EchoNova Audio\nDocument Title

#### Generation

In [10]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model="deepseek-chat", temperature=0)
rag_prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [11]:
chain.invoke("What is the company name?")

'The company name is **EchoNova Audio**. It is a multinational retailer specializing in premium audio electronics and related services. The company partners with global distributors and serves end users directly.'